In [1]:
import pandas as pd
import numpy as np

In [4]:
import pickle
benchmark_1 = pickle.load( open( "./Benchmark_Search/benchmark_df.pkl", "rb" ) )
benchmark_2 = pickle.load( open( "./Benchmark_Search/benchmark_df_Friday_1.pkl", "rb" ) )
benchmark_3 = pickle.load( open( "./Benchmark_Search/benchmark_df_Friday_2.pkl", "rb" ) )

In [6]:
print(benchmark_1)
print(benchmark_2)
print(benchmark_3)

                 test_rmse  fit_time  test_time
Algorithm                                      
NormalPredictor   1.548175  1.037216   2.900593
                 test_rmse   fit_time  test_time
Algorithm                                       
NMF               1.004073  43.303196   3.646530
SlopeOne          1.010384   1.393850   9.716020
BaselineOnly      1.011109   2.904197   3.869235
CoClustering      1.131746  30.726416   6.051676
NormalPredictor   1.549493   1.068971   3.377020
           test_rmse  fit_time  test_time
Algorithm                                
SVD         1.036598  69.08436   5.520285


In [11]:
grid = pd.read_csv('./Data/final_game_user_grid_06_07_19.csv')

In [12]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387492 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1387492 non-null object
game_name       1385813 non-null object
game_genres     1383564 non-null object
language        1387490 non-null object
started_at      1387492 non-null object
viewer_count    1387492 non-null int64
max             1387492 non-null int64
dtypes: int64(2), object(5)
memory usage: 74.1+ MB


In [13]:
grid = grid.dropna()

In [14]:
grid = grid.dropna(how='any',axis=0) 

In [15]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1383562 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1383562 non-null object
game_name       1383562 non-null object
game_genres     1383562 non-null object
language        1383562 non-null object
started_at      1383562 non-null object
viewer_count    1383562 non-null int64
max             1383562 non-null int64
dtypes: int64(2), object(5)
memory usage: 84.4+ MB


In [16]:
max_value_username = pd.DataFrame(grid.groupby('user_name')['max'].max().reset_index())

In [17]:
max_val_dict = max_value_username.groupby('user_name')['max'].apply(list).to_dict()

In [18]:
grid['max_game'] = grid['user_name'].map(max_val_dict)

In [19]:
grid['max_game_int'] = grid.max_game.str[0].astype(int)
grid = grid.drop('max_game', axis = 1)

In [20]:
grid['score'] = grid['max']/grid['max_game_int']

,user_name,game_name,game_genres,language,started_at,viewer_count,max,max_game_int,score
1122503,Chisaihato,Apex Legends,FPS,en,2019-06-03 05:00:25,16,16,62,0.258065
1254963,Anime_Boyfriend,Apex Legends,Shooter,en,2019-06-07 00:58:26,30,30,30,1.000000
174748,Thrash2469,Counter-Strike: Global Offensive,FPS,en,2019-06-04 00:30:24,11,11,11,1.000000


In [22]:
from sklearn.preprocessing import minmax_scale
grid['scaled_score'] = minmax_scale(grid['score'], feature_range=(1, 5))

In [23]:
grid = grid.dropna()

In [26]:
grid.groupby('user_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,user_name,scaled_score
27473,Fonbet_RocketLeague,378
77300,StreamerHouse,375
123375,luke4316live,333
110114,gaules,315
27472,Fonbet_RLH,270


In [27]:
grid.groupby('game_genres')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_genres,scaled_score
0,Action,211633
29,Shooter,165931
7,FPS,131286
24,RPG,118595
15,MMORPG,88901


In [28]:
grid.groupby('game_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_name,scaled_score
128,Grand Theft Auto V,37860
180,Minecraft,35715
243,Rocket League,35043
37,Black Desert Online,33723
97,Escape From Tarkov,33366


In [29]:
min_number_scores = 5
filter_users = grid['user_name'].value_counts() > min_number_scores
filter_users = filter_users[filter_users].index.tolist()

In [30]:
grid_new = grid[(grid['user_name'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(grid.shape))
print('The new data frame shape:\t{}'.format(grid_new.shape))

The original data frame shape:	(1364027, 10)
The new data frame shape:	(1137673, 10)


In [32]:
import pickle

In [33]:
pickle.dump(grid_new, open("./Data/final_grid_06_07_19.pkl", "wb" ) )

In [34]:
grid_new = pickle.load( open( "./Data/final_grid_06_07_19.pkl", "rb" ) )

In [35]:
# Start here to work on the recommender app script, after loading the final pickle file.

In [7]:
grouped = grid_new.groupby(by = ['user_name', 'game_name'])['scaled_score'].agg([np.mean])

In [8]:
grouped.columns = grouped.columns.map(''.join)

In [9]:
grouped = grouped.reset_index()

In [10]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89975 entries, 0 to 89974
Data columns (total 3 columns):
user_name    89975 non-null object
game_name    89975 non-null object
mean         89975 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [11]:
import surprise
from surprise import Dataset, accuracy, Reader
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split

In [12]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(grouped[['user_name', 'game_name', 'mean']], reader)

In [13]:
from surprise import NormalPredictor, BaselineOnly, SVD, SVDpp, SlopeOne, NMF, KNNBaseline,KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering

In [42]:
benchmark = []
# Iterate over all algorithms
for algorithm in [NormalPredictor(), BaselineOnly()]:


    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.794825,0.219064,0.191618
NormalPredictor,1.144264,0.079577,0.167529


In [43]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.79536384, 0.79510842, 0.79171042]),
 'fit_time': (0.12734484672546387, 0.11901283264160156, 0.1282482147216797),
 'test_time': (0.15812921524047852, 0.14609718322753906, 0.24782180786132812)}

In [46]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.7970


0.7969788038151487

In [47]:
grouped[grouped['user_name']=='sentynalol']

,user_name,game_name,mean
76794,sentynalol,The Elder Scrolls V: Skyrim,3.333333


In [ ]:
#dump.dump('./dump_file', predictions, algo)
#predictions, algo = dump.load('./dump_file')

####  Predicting Genres for Streamers (user-based similarities) ####

In [48]:
from collections import defaultdict

In [49]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [50]:
predictions

[Prediction(uid='CarritaBurrita', iid='Music & Performing Arts', r_ui=3.6311111111111107, est=3.5904659569365407, details={'was_impossible': False}),
 Prediction(uid='strakatv', iid='Counter-Strike: Global Offensive', r_ui=3.5794238683127575, est=3.557218460638966, details={'was_impossible': False}),
 Prediction(uid='TheWrongCast', iid='Dauntless', r_ui=4.333333333333334, est=3.6965885758596357, details={'was_impossible': False}),
 Prediction(uid='b1ue_dragon', iid='Heroes of Might and Magic III: The Shadow of Death', r_ui=4.0, est=3.2565701019450937, details={'was_impossible': False}),
 Prediction(uid='OlMahk', iid='Mordhau', r_ui=3.5, est=3.342174100850436, details={'was_impossible': False}),
 Prediction(uid='gaussmath_', iid='DayZ', r_ui=5.0, est=3.655641149556071, details={'was_impossible': False}),
 Prediction(uid='fiqshn', iid='Dark Souls III', r_ui=3.0, est=3.5136210823828677, details={'was_impossible': False}),
 Prediction(uid='Biohazard_Acid', iid='Rust', r_ui=3.0, est=3.54953

In [52]:
iids = grouped['game_name'].unique()

In [53]:
iids_sentynalol = grid_new.loc[grid_new['user_name']=='sentynalol', 'user_name']

In [54]:
iids_to_predict = np.setdiff1d(iids, iids_sentynalol)

In [55]:
testset_personal = [['sentynalol', iid, 4.] for iid in iids_to_predict]

In [56]:
predictions = algo.test(testset_personal)

In [57]:
predictions

[Prediction(uid='sentynalol', iid='7 Days to Die', r_ui=4.0, est=3.383923904363135, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='A Hat in Time', r_ui=4.0, est=3.7376464047778755, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='A Plague Tale: Innocence', r_ui=4.0, est=3.0748324709139494, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='A Way Out', r_ui=4.0, est=3.518719457240425, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='ARK', r_ui=4.0, est=3.2295917237545533, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='ASMR', r_ui=4.0, est=3.2637624689688667, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='ATLAS', r_ui=4.0, est=3.35657864019659, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='Agar.io', r_ui=4.0, est=3.2374988916039555, details={'was_impossible': False}),
 Prediction(uid='sentynalol', iid='Age of Empires II', r_ui=4.

In [58]:
top_n = get_top_n(predictions, n=10)

In [59]:
get_top_n(predictions)

defaultdict(list,
            {'sentynalol': [("Don't Starve Together", 4.062730209369868),
              ("Five Nights at Freddy's VR: Help Wanted", 4.04459013935462),
              ('Doki Doki Literature Club', 3.9828892560019975),
              ('Team Fortress 2', 3.9417571970884384),
              ('The Elder Scrolls IV: Oblivion', 3.9402543114179247),
              ('Silent Hill', 3.9290787074388165),
              ('Minecraft', 3.9191465194208197),
              ('Tabletop RPGs', 3.8991092827698703),
              ('Danganronpa: Trigger Happy Havoc', 3.898824006831341),
              ('Diablo', 3.8697621640563042)]})

In [60]:
for uid, user_ratings in top_n.items():
    print(uid, set([iid for (iid, _) in user_ratings]))

sentynalol {'Danganronpa: Trigger Happy Havoc', 'The Elder Scrolls IV: Oblivion', 'Minecraft', 'Tabletop RPGs', 'Team Fortress 2', 'Doki Doki Literature Club', "Don't Starve Together", "Five Nights at Freddy's VR: Help Wanted", 'Silent Hill', 'Diablo'}


#### Predicting Similar Genres Based on Current Genre (item-based similarity) #### 

In [14]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir


In [15]:
game_grouped = grid_new.groupby(by = ['game_name', 'user_name'])['scaled_score'].agg([np.mean])

In [16]:
game_grouped = game_grouped.reset_index()

In [17]:
game_grouped.sample(10)

,game_name,user_name,mean
46355,Monster Hunter World,khnix,3.900000
75532,The Last of Us,GorilaNerd,3.077969
18216,Dauntless,its_MDGaming,3.588235
10970,Call of Duty: Black Ops 4,El_Seb,5.000000
60853,Rocket League,LowKeyTiNo,5.000000
23755,Destiny 2,QuickPIay,4.000000
69763,Super Mario Maker,SparkWulfe,4.000000
81627,Total War: Three Kingdoms,fabssik4,3.285714
64231,Science & Technology,Trenton_Pottruff,3.533333
63340,Rust,__소한__,4.000000


In [37]:
game_data = Dataset.load_from_df(game_grouped[['user_name', 'game_name', 'mean']], reader)

In [38]:
trainset = game_data.build_full_trainset()

In [40]:
# First, train the algortihm to compute the similarities between items
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0, 'user_based': False  # no shrinkage
               }
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [47]:
inner = algo.trainset.to_inner_iid('The Elder Scrolls V: Skyrim')

In [48]:
# Retrieve inner ids of the nearest neighbors of the game in question.
game_neighbors = algo.get_neighbors(inner, k=10)

In [49]:
game_neighbors

[2, 29, 24, 178, 181, 177, 229, 70, 15, 325]

In [50]:
games = [algo.trainset.to_raw_iid(iiid) for iiid in game_neighbors]

In [51]:
print('The 10 nearest neighbors of this game are:')
for game in games:
    print(game)

The 10 nearest neighbors of this game are:
A Plague Tale: Innocence
Battalion 1944
Auto Chess
Mordhau
Music & Performing Arts
Monster Hunter World
Retro
Deathgarden
Apex Legends
VRChat
